In [19]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool, InjectedToolArg
from langchain_core.messages import HumanMessage, ToolMessage
import requests
from dotenv import load_dotenv
from typing import Annotated

In [20]:
load_dotenv()

EXCHANGE_RATE_API_KEY = os.getenv("EXCHANGE_RATE_API_KEY")

In [21]:
# Tool creation
@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
    """
    Fetches the conversion from base currency to target currency.
    :param base_currency: The base currency.
    :param target_currency: The target currency in which conversion is made.
    :return: Conversion factor.
    """
    url = f"https://v6.exchangerate-api.com/v6/{EXCHANGE_RATE_API_KEY}/pair/{base_currency}/{target_currency}"
    response = requests.get(url).json()
    conversion_rate = response.get("conversion_rate", 0.00)
    return float(conversion_rate)

@tool
def convert(base_currency_value: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
    """
    Given the conversion rate, it converts base currency value to target currency value.
    :param base_currency_value: Amount to be converted.
    :param conversion_rate: Conversion rate.
    :return: The converted amount.
    """
    return base_currency_value * conversion_rate

In [22]:
# Tool binding
llm = ChatOpenAI(model="gpt-5-nano")

llm_with_tools = llm.bind_tools([get_conversion_factor, convert])

In [23]:
messages = [HumanMessage("What is the conversion factor between USD and INR, and based on that can you convert 10 USD to INR?")]

In [24]:
ai_message = llm_with_tools.invoke(messages)

In [25]:
ai_message

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_1b4pIppM9TqPKItwMIMWQyqn', 'function': {'arguments': '{"base_currency":"USD","target_currency":"INR"}', 'name': 'get_conversion_factor'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 543, 'prompt_tokens': 244, 'total_tokens': 787, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 512, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CnqfkDf86gvvMiV6AVXaCgST4NTZ3', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b2d99-4a2d-7042-bb94-82682f018ac5-0', tool_calls=[{'name': 'get_conversion_factor', 'args': {'base_currency': 'USD', 'target_currency': 'INR'}, 'id': 'call_1b4pIppM9TqPKItwMIMWQyqn', 'type': 'tool_call'}], usage_metadata={'input_token

In [26]:
messages.append(ai_message)

In [27]:
tool_calls = ai_message.tool_calls

In [28]:
tool_calls

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'USD', 'target_currency': 'INR'},
  'id': 'call_1b4pIppM9TqPKItwMIMWQyqn',
  'type': 'tool_call'}]

In [29]:
tool_message_1 = get_conversion_factor.invoke(tool_calls[0])

In [30]:
tool_message_1

ToolMessage(content='91.028', name='get_conversion_factor', tool_call_id='call_1b4pIppM9TqPKItwMIMWQyqn')

In [31]:
messages.append(tool_message_1)

In [35]:
# Send this back to LLM
llm_forced_convert = llm.bind_tools([get_conversion_factor, convert], tool_choice="convert")
ai_message_2 = llm_forced_convert.invoke(messages)

In [37]:
messages.append(ai_message_2)

In [42]:
tool_call_2 = ai_message_2.tool_calls[0]

In [43]:
tool_call_2

{'name': 'convert',
 'args': {'base_currency_value': 10},
 'id': 'call_GEe9RIfHI2Cy2EH4nmduOvIj',
 'type': 'tool_call'}

In [44]:
conversion_rate = float(tool_message_1.content)
conversion_rate

91.028

In [45]:
tool_call_2['args']['conversion_rate'] = conversion_rate
tool_message_2 = convert.invoke(tool_call_2)
tool_message_2

ToolMessage(content='910.2800000000001', name='convert', tool_call_id='call_GEe9RIfHI2Cy2EH4nmduOvIj')

In [46]:
messages.append(tool_message_2)

In [47]:
messages

[HumanMessage(content='What is the conversion factor between USD and INR, and based on that can you convert 10 USD to INR?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_1b4pIppM9TqPKItwMIMWQyqn', 'function': {'arguments': '{"base_currency":"USD","target_currency":"INR"}', 'name': 'get_conversion_factor'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 543, 'prompt_tokens': 244, 'total_tokens': 787, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 512, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CnqfkDf86gvvMiV6AVXaCgST4NTZ3', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b2d99-4a2d-7042-bb94-82682f018ac5-0', tool_calls=[{'name': 'get

In [48]:
llm_with_tools.invoke(messages)

AIMessage(content='- Conversion factor: 1 USD = 91.028 INR\n- 10 USD in INR: 910.28 INR\n\nNote: exchange rates fluctuate. If you’d like, I can fetch the latest rate or convert a different amount.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 379, 'prompt_tokens': 326, 'total_tokens': 705, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 320, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-Cnqqy3QUBAa7x7o5aeEcEXGZbEg9T', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b2da3-ec66-7752-88d1-3a4ae8c97a58-0', usage_metadata={'input_tokens': 326, 'output_tokens': 379, 'total_tokens': 705, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 320}})